In [1]:
import torch
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, dataframe, text_column):
        self.texts = dataframe[text_column].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]#f"query: {self.texts[idx]}"

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [3]:
data = pd.read_csv("../data/cleared_v2/cleared_v2.csv")
data

,target,source
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёрув...",Мы никогда не забудем этого честного человека.
...,...,...
79928,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
79929,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
79930,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
79931,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [4]:
BATCH_SIZE = 128

In [5]:
dataset = TextDataset(data, 'source')
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
model = model.to('cuda')
model.eval();

In [6]:
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**{k: v.to('cuda') for k, v in batch_dict.items()})
embeddings = average_pool(outputs.last_hidden_state.cpu(), batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[90.81391906738281, 72.1339111328125], [70.5354232788086, 88.76107788085938]]


In [ ]:
all_embeddings = []

with torch.no_grad():
    for batch in tqdm(loader):
        batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
        batch_dict = {k: v.to('cuda') for k, v in batch_dict.items()}
        
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        
        all_embeddings.append(embeddings.cpu().numpy().copy())

  0%|          | 0/1249 [00:00<?, ?it/s]

# rubert-tiny-turbo

In [6]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sergeyzh/rubert-tiny-turbo', device='cuda')
model.eval()

sentences = ["привет мир", "hello world", "здравствуй вселенная"]
embeddings = model.encode(sentences)
print(util.dot_score(embeddings, embeddings))

tensor([[1.0000, 0.8918, 0.9049],
        [0.8918, 1.0000, 0.8783],
        [0.9049, 0.8783, 1.0000]])


In [7]:
sentences = ["вода лилась рекой", "камень стоял под столом"]
embeddings = model.encode(sentences)
print(util.dot_score(embeddings, embeddings))

tensor([[1.0000, 0.6697],
        [0.6697, 1.0000]])


In [7]:
all_embeddings = []
for batch in tqdm(loader):
    all_embeddings.append(model.encode(batch).copy())

  0%|          | 0/625 [00:00<?, ?it/s]

In [14]:
all_embeddings = np.concatenate(all_embeddings, axis=0)

# data from kaggle

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModel
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import h5py
from sentence_transformers import SentenceTransformer
from ast import literal_eval
from copy import deepcopy
from collections import Counter

pd.set_option('display.max_colwidth', 100)

In [2]:
data = pd.read_pickle('../data/data_with_scores.pkl')
data

,target,source,similarities
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат вермгыг варункв,Те мальчики не выполнят задание в назначенный срок.,"{0: 1.0, 52122: 0.9042518734931946, 18930: 0.8891035318374634, 41202: 0.8863261342048645, 6959: ..."
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала.","{1: 0.9999999403953552, 4463: 0.9709538817405701, 40149: 0.9473190307617188, 10594: 0.9426226615..."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах,"{2: 0.9999999403953552, 33249: 0.9213534593582153, 11802: 0.9161194562911987, 27806: 0.907817661..."
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит.","{3: 0.9999997019767761, 12914: 0.9601884484291077, 4014: 0.9214043021202087, 62795: 0.9111366868..."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёрувлвылув",Мы никогда не забудем этого честного человека.,"{4: 1.0, 55368: 0.9368814826011658, 61451: 0.9308530688285828, 70427: 0.9243783950805664, 58062:..."
...,...,...,...
77369,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄хматын ма̄гыс лю̄льсаӈыг сусхатуӈкве ве̄рме̄гыт.,"Мне кажется, что подобные случаи могут вызывать подозрение.","{77369: 1.0000001192092896, 75988: 0.9060379862785339, 75732: 0.8967535495758057, 40617: 0.89106..."
77370,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.,"{77370: 1.0000001192092896, 25801: 0.9226779341697693, 25023: 0.9087227582931519, 48245: 0.90375..."
77371,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления меда нет.","{77371: 0.9999998211860657, 26336: 0.8946378231048584, 26326: 0.8911178112030029, 48607: 0.89045..."
77372,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэнк Заппа ёт гастролин минас.","Затем Уэйтс отправился на гастроли с такими музыкантами, как Чарли Рич и Фрэнк Заппа.","{77372: 0.9999999403953552, 31706: 0.8608967065811157, 53618: 0.8565278053283691, 66384: 0.85216..."


In [3]:
parsed_corpus = pd.read_pickle('../data/parsed_corpus_changed.pkl')
parsed_corpus

,source,target,link
0,[В июне в этнографическом музее «Торум Маа» города Ханты-Мансийска дети посещали лагерь «Таксар ...,[Лӯпта э̄тпост Ханты-Мансийск ӯст «То̄рум Маа» музейт на̄врамыт ат хо̄тал ханищтахтасыт. Ты тэ...,https://khanty-yasang.ru//luima-seripos/no-14-1296/16763
1,"[Демид Чамалтынов, 9 лет, – Я живу в Ханты-Мансийске, обучаюсь в школе № 2. В этом году я впервы...","[Демид Чамалтынов, 9 та̄л, – «Та̄ксар ма̄хум» – «Крепкие люди» сменан ам ты та̄л о̄выл щёс ёхтыс...",https://khanty-yasang.ru//luima-seripos/no-16-1298/16923
2,[В Берёзовском районе в семи километрах ниже от посёлка Сосьва и в пятнадцати километрах от Усть...,[Ха̄льӯс районт Кульпас па̄вылныл ло̄ӈхаль са̄т ве̄рста Та̄гт а̄ хосыт ма̄нь ма̄ньщи па̄выл – С...,https://khanty-yasang.ru//luima-seripos/no-17-1275/15058
3,[Я хочу написать про своих детей. Этот год для нас был очень важным. Мои дети Коля и Надя –два в...,"[Ам амки на̄врама̄гум урыл хансуӈкв таӈхе̄гум. Ты та̄л ам ма̄гсылум тамле о̄лыс, ам кит на̄врама...",https://khanty-yasang.ru//luima-seripos/no-13-1295/16691
4,[Каждое лето Ханты-Мансийский этнографический музей под открытым небом «Торум Маа» для маленьких...,"[Ка̄сыӈ туи пора Ханты-Мансийск ӯст о̄лнэ на̄врамыт магыс «То̄рум Маа» музеит ханищтап ва̄раве,...",https://khanty-yasang.ru//luima-seripos/no-17-1299/17003
...,...,...,...
3745,[Многие молодые ребята из числа коренных малочисленных народов Севера после окончания школы охот...,[Ханты-Мансийск ӯст акв ма̄ньщи пыг о̄лы. Аквматнакт ам тав уре̄т хансыгла̄лсум. Тав наме Алекс...,https://khanty-yasang.ru//luima-seripos/no-4-1214/10040
3746,"[С хантыйской мастерицей Надеждой Алексеевной Гришкиной я познакомилась в музее «Торум Маа», где...",[Надежда Алексеевна Гришкина ты та̄л атпан нупыл ат та̄лэ то̄влыс. Нэ̄ ще̄мьятэ ёт Тугияны па̄вы...,https://khanty-yasang.ru//luima-seripos/no-24-1186/8519
3747,[Для жителей сельского поселения села Саранпауль традиционный праздник «День оленевода» - один и...,"[Ма̄н округувт са̄в са̄лыт туп тит ма̄т о̄ньщавет, ты Ха̄льӯс район Саранпа̄вылт ос Белоярский ...",https://khanty-yasang.ru//luima-seripos/no-5-1167/7614
3748,[Альбина Волосовская уже много лет живет и работает в п. Октябрьском. Сама она родом из маленько...,[Ты хурит Октябрьский па̄вылт о̄лнэ ма̄ньщи нэ̄ Альбина Петровна Волосовская по̄слым о̄лы. Такви...,https://khanty-yasang.ru//luima-seripos/no-14-1056/896


In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
sentences = ["This is an example sentence", "Each sentence is converted"]

labse = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = labse.encode(sentences)
print(embeddings)

C:\Users\1645286\venvs\py310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[[ 0.02882476 -0.00602382 -0.05947006 ... -0.0300225  -0.02960701
   0.00067479]
 [-0.05550231  0.02546487 -0.02157258 ...  0.02932104  0.01150043
  -0.00848787]]


C:\Users\1645286\venvs\py310\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
all_corpus = data.target.tolist() + parsed_corpus.target.apply(lambda x: '\n'.join(x)).tolist()

In [6]:
data.source.loc[1], data.target.loc[1]

('Бегая к берегу озера пришла, к воде подбежала.',
 'Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.')

In [7]:
print(labse.tokenizer.convert_ids_to_tokens(labse.tokenizer.encode(data.target.loc[1])))

['[CLS]', 'Ха', '##̄', '##йт', '##ыма', '##тэ', 'ту', '##̄', '##р', 'ва', '##̄', '##тан', 'ё', '##хт', '##ыс', ',', 'ви', '##т', 'ва', '##̄', '##тан', 'ха', '##̄', '##йт', '##ыс', '.', '[SEP]']


In [8]:
print(labse.tokenizer.convert_ids_to_tokens(labse.tokenizer.encode(data.source.loc[1])))

['[CLS]', 'Бе', '##га', '##я', 'к', 'берегу', 'озера', 'пришла', ',', 'к', 'воде', 'под', '##бежал', '##а', '.', '[SEP]']


In [9]:
len(labse.tokenizer)

501153

In [10]:
old_tokenizer = deepcopy(labse.tokenizer)
old_tokenizer

BertTokenizerFast(name_or_path='sentence-transformers/LaBSE', vocab_size=501153, model_max_length=256, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## add new tokens to tokenizer

In [28]:
word_freqs = Counter()

for text in tqdm(all_corpus):
    words_with_offsets = labse.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        if len(word) > 1:
            word_freqs[word] += 1

  0%|          | 0/81124 [00:00<?, ?it/s]

In [20]:
word_freqs.most_common(20)

[('ос', 38228),
 ('ты', 17358),
 ('та', 14719),
 ('ат', 14211),
 ('ма̄хум', 13972),
 ('Ты', 12616),
 ('тав', 11556),
 ('та̄л', 10314),
 ('о̄лнэ', 9334),
 ('урыл', 9088),
 ('о̄с', 9064),
 ('са̄в', 8994),
 ('порат', 8686),
 ('ань', 8419),
 ('тот', 8407),
 ('та̄н', 8273),
 ('ма̄н', 8049),
 ('хо̄тпат', 7991),
 ('ёт', 7825),
 ('Ань', 7633)]

In [21]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¬', '¸', '×', 'ä', 'é', 'ó', 'ö', 'ü', '̄', 'Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ѐ', 'ё', 'є', 'ў', 'ӆ', 'Ӈ', 'ӈ', 'ӊ', 'ӑ', 'ә', '№', '™']


In [22]:
vocab = deepcopy(alphabet)

In [55]:
init_splits = {word: [c for c in word] for word in word_freqs.keys()}
dict(zip(
    list(init_splits.keys())[:5],
    list(init_splits.values())[:5]
))

{'Та': ['Т', 'а'],
 'пыгрисит': ['п', 'ы', 'г', 'р', 'и', 'с', 'и', 'т'],
 'маим': ['м', 'а', 'и', 'м'],
 'вармаль': ['в', 'а', 'р', 'м', 'а', 'л', 'ь'],
 'э̄рнэ': ['э', '̄', 'р', 'н', 'э']}

In [24]:
def compute_pair_freqs(splits, word_freqs, disable_tqdm=False):
    pair_freqs = Counter()
    for word, freq in tqdm(word_freqs.items(), disable=disable_tqdm):
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [57]:
init_pair_freqs = compute_pair_freqs(init_splits, word_freqs)

  0%|          | 0/104834 [00:00<?, ?it/s]

In [58]:
init_pair_freqs.most_common(10)

[(('а', '̄'), 354565),
 (('т', 'а'), 231852),
 (('о', '̄'), 173997),
 (('т', 'ы'), 153771),
 (('а', 'н'), 149215),
 (('м', 'а'), 147474),
 (('ы', 'л'), 140337),
 (('ы', 'т'), 132754),
 (('̄', 'л'), 130946),
 (('̄', 'н'), 121158)]

In [27]:
def merge_pair(a, b, splits, word_freqs, disable_tqdm=False):
    for word in tqdm(word_freqs, disable=disable_tqdm):
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [28]:
vocab_size = 5000
merges = {}
old_vocab_size = len(vocab)

splits = {word: [c for c in word] for word in word_freqs.keys()}

for _ in tqdm(range(vocab_size - old_vocab_size)):
    pair_freqs = compute_pair_freqs(splits, word_freqs, True)
    best_pair, _ = pair_freqs.most_common(1)[0]
    splits = merge_pair(*best_pair, splits, word_freqs, True)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

  0%|          | 0/4853 [00:00<?, ?it/s]

In [30]:
with open('merges.pkl', 'wb') as f:
    pickle.dump(merges, f)
with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [32]:
len(vocab)

5000

In [34]:
new_tokenizer = deepcopy(old_tokenizer)

In [51]:
new_tokens = list(set(vocab[:1000]) - set(new_tokenizer.vocab.keys()))
new_tokens[:5]

['ӯсн', 'лё̄', 'тамле', 'ма̄ныл', 'са̄вит']

In [52]:
len(new_tokens)

613

In [62]:
new_tokenizer.add_tokens(new_tokens)

613

In [65]:
data.target.loc[1]

'Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.'

In [63]:
print(new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(data.target.loc[1])))

['[CLS]', 'Ха̄', 'йт', 'ы', '##м', 'атэ', 'тӯ', 'р', 'ва̄т', 'ан', 'ёхтыс', ',', 'вит', 'ва̄т', 'ан', 'ха̄й', 'т', 'ыс', '.', '[SEP]']


In [64]:
print(new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(data.source.loc[1])))

['[CLS]', 'Бе', '##га', '##я', 'к', 'берегу', 'озера', 'пришла', ',', 'к', 'воде', 'под', '##бежал', '##а', '.', '[SEP]']


# modified tokenizer

In [73]:
old_tokenizer = deepcopy(labse.tokenizer)
old_tokenizer

BertTokenizerFast(name_or_path='sentence-transformers/LaBSE', vocab_size=501153, model_max_length=256, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## bpe train (not correct)

In [74]:
word_freqs.most_common(10)

[('ос', 38228),
 ('ты', 17358),
 ('та', 14719),
 ('ат', 14211),
 ('ма̄хум', 13972),
 ('Ты', 12616),
 ('тав', 11556),
 ('та̄л', 10314),
 ('о̄лнэ', 9334),
 ('урыл', 9088)]

In [76]:
alphabet = []

for word in tqdm(word_freqs.keys()):
    for i, letter in enumerate(word):
        if i == 0:
            if letter not in alphabet:
                alphabet.append(letter)
        else:
            if '##' + letter not in alphabet:
                alphabet.append('##' + letter)
                
alphabet.sort()

print(alphabet)

  0%|          | 0/104834 [00:00<?, ?it/s]

['##0', '##1', '##2', '##3', '##4', '##5', '##6', '##7', '##8', '##9', '##A', '##B', '##C', '##D', '##E', '##F', '##G', '##H', '##I', '##J', '##K', '##L', '##M', '##N', '##O', '##P', '##R', '##S', '##T', '##U', '##V', '##W', '##X', '##Y', '##Z', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##q', '##r', '##s', '##t', '##u', '##v', '##w', '##x', '##y', '##z', '##¬', '##¸', '##×', '##ä', '##é', '##ó', '##ö', '##ü', '##̄', '##Ё', '##А', '##Б', '##В', '##Г', '##Д', '##Е', '##Ж', '##З', '##И', '##Й', '##К', '##Л', '##М', '##Н', '##О', '##П', '##Р', '##С', '##Т', '##У', '##Ф', '##Х', '##Ц', '##Ч', '##Ш', '##Щ', '##Ы', '##Ь', '##Э', '##Ю', '##Я', '##а', '##б', '##в', '##г', '##д', '##е', '##ж', '##з', '##и', '##й', '##к', '##л', '##м', '##н', '##о', '##п', '##р', '##с', '##т', '##у', '##ф', '##х', '##ц', '##ч', '##ш', '##щ', '##ъ', '##ы', '##ь', '##э', '##ю', '##я', '##ѐ', '##ё', '##є', '##ў', '##ӆ', '##Ӈ', '##ӈ', '##ӊ', '##ӑ'

In [77]:
vocab = deepcopy(alphabet)

In [98]:
def get_splits_v2(word_freqs, disable_tqdm=False):
    splits = {}
    for word in tqdm(word_freqs, disable=disable_tqdm):
        l = ['##' + c for c in word]
        l[0] = word[0]
        splits[word] = deepcopy(l)
    return splits
splits = get_splits_v2(word_freqs)

  0%|          | 0/104834 [00:00<?, ?it/s]

In [95]:
dict(zip(
    list(splits.keys())[:5],
    list(splits.values())[:5]
))

{'Та': ['Т', '##а'],
 'пыгрисит': ['п', '##ы', '##г', '##р', '##и', '##с', '##и', '##т'],
 'маим': ['м', '##а', '##и', '##м'],
 'вармаль': ['в', '##а', '##р', '##м', '##а', '##л', '##ь'],
 'э̄рнэ': ['э', '##̄', '##р', '##н', '##э']}

In [99]:
def compute_pair_freqs_v2(splits, word_freqs, disable_tqdm=False):
    pair_freqs = Counter()
    for word, freq in tqdm(word_freqs.items(), disable=disable_tqdm):
        split = splits[word]
        if len(split) == 1:
            continue

        for i in range(len(split) - 2):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [100]:
def merge_pair_v2(a, b, splits, word_freqs, disable_tqdm=False):
    for word in tqdm(word_freqs, disable=disable_tqdm):
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                if i == 0:
                    split = split[:i] + [a.replace('##', '') + b.replace('##', '')] + split[i + 2:]
                else:
                    split = split[:i] + ['##' + a.replace('##', '') + b.replace('##', '')] + split[i + 2:]
            else:
                i += 1
                
        splits[word] = split
    return splits

In [101]:
additional_vocab_size = 100
merges = {}

splits = get_splits_v2(word_freqs, True)

for _ in tqdm(range(additional_vocab_size)):
    pair_freqs = compute_pair_freqs_v2(splits, word_freqs, True)
    best_pair, _ = pair_freqs.most_common(1)[0]
    splits = merge_pair_v2(*best_pair, splits, word_freqs, True)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

  0%|          | 0/100 [00:00<?, ?it/s]

## wordpiece training

In [193]:
word_freqs.most_common(10)

[('ос', 38228),
 ('ты', 17358),
 ('та', 14719),
 ('ат', 14211),
 ('ма̄хум', 13972),
 ('Ты', 12616),
 ('тав', 11556),
 ('та̄л', 10314),
 ('о̄лнэ', 9334),
 ('урыл', 9088)]

In [80]:
len([text for text in tqdm(all_corpus) if old_tokenizer.unk_token_id in old_tokenizer.encode(text)])

  0%|          | 0/81124 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 256). Running this sequence through the model will result in indexing errors


33371

In [64]:
def add_tokens(vocab):
    new_tokenizer = deepcopy(labse.tokenizer)
    new_tokens = list(set(vocab) - set(new_tokenizer.vocab.keys()))
    num_tokens_added = new_tokenizer.add_tokens(new_tokens)
    print(f'Added {num_tokens_added} new tokens to tokenizer')
    return new_tokenizer, new_tokens

In [204]:
new_tokenizer, new_tokens = add_tokens(['ханищ##та'])

['ханищ##та']
['ханищта']
Added 1 new tokens to tokenizer


In [207]:
{k: v for k, v in new_tokenizer.vocab.items() if 'ханищта' in k}

{'ханищта': 501153}

In [39]:
from tokenizers.trainers import WordPieceTrainer
from tokenizers import models, pre_tokenizers, decoders, Tokenizer

In [52]:
new_wordpiece = Tokenizer(models.WordPiece())
new_wordpiece.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
#new_wordpiece.decoder = decoders.ByteLevel()
trainer = WordPieceTrainer(
    vocab_size=1500,
    min_frequency=30,
    show_progress=True,
    continuing_subword_prefix='##'
)

In [192]:
new_wordpiece.train_from_iterator(all_corpus, trainer=trainer, length=len(all_corpus))

In [54]:
sorted(new_wordpiece.get_vocab(), key=lambda x: len(x), reverse=True)

['ханищтахтуӈкве',
 'рӯпитаӈкве',
 'э̄лумхо̄лас',
 'рӯпитэ̄гыт',
 'ханищтахтын',
 'потыртаӈкве',
 'ханищтахтас',
 '##лумхо̄лас',
 'Саранпа̄выл',
 'хо̄нтхатыг',
 'о̄ньще̄гыт',
 'ве̄рме̄гыт',
 '##таве̄сыт',
 '##хатэ̄гыт',
 'потыртасыт',
 '##лаве̄сыт',
 '##има̄гыс',
 'та̄рвитыӈ',
 'ва̄руӈкве',
 'ва̄ре̄гыт',
 'ва̄рмалит',
 '##щхӣпыӈ',
 'таима̄гыс',
 'та̄нанылн',
 'па̄вылныл',
 'ня̄врамыт',
 '##хуйплов',
 'ёхталасыт',
 'ма̄хманув',
 'Александр',
 'суссылтап',
 'на̄врамыт',
 'хансуӈкве',
 'хо̄талэ̄т',
 'рӯпитас',
 'ва̄рмаль',
 '##луӈкве',
 'пормасыт',
 'са̄ӈквыл',
 'ханищтап',
 '##а̄врам',
 'хо̄талыт',
 'о̄лэ̄гыт',
 '##лаӈкве',
 'Светлана',
 '##та̄гыл',
 '##туӈкве',
 'ма̄ньлат',
 '##тэ̄гыт',
 '##е̄ккар',
 '##ве̄сыт',
 'о̄вылтах',
 'нэ̄пакыт',
 'Владимир',
 'Ха̄льӯс',
 '##лэ̄гыт',
 '##тыглас',
 '##па̄выл',
 'пӯльниц',
 'Мансийск',
 '##хо̄тал',
 'та̄наныл',
 'солко̄ви',
 '##тыглах',
 '##таӈкве',
 'рӯпитан',
 'ищхӣпыӈ',
 'ханищтах',
 'ханищтан',
 'потыртас',
 'ӯргалан',
 'о̄вылтыт',
 '

In [195]:
new_tokenizer, new_tokens = add_tokens(list(new_wordpiece.get_vocab().keys()))

Added 767 new tokens to tokenizer


In [141]:
TKN_DIR_OLD = 'labse_tokenizer'
TKN_DIR_NEW = 'new_tokenizer'

In [92]:
def find_conflicting_tokens(new_tokens, old_vocab):
    conflicting_tokens = []
    
    for token in tqdm(new_tokens):
        if token.startswith("##"):
            # Убедимся, что корень не существует в старом словаре
            root_token = token[2:]  # Убираем '##'
            if root_token in old_vocab:
                conflicting_tokens.append(root_token)
        else:
            # Проверяем, что не существует подслов с этим корнем
            subword_token = f"##{token}"
            if subword_token in old_vocab:
                conflicting_tokens.append(subword_token)
    
    return conflicting_tokens

In [93]:
def remove_conflicting_tokens(conflicting_tokens, old_vocab):
    cleaned_vocab = deepcopy(old_vocab)
    
    for token in conflicting_tokens:
        if token in cleaned_vocab:
            del cleaned_vocab[token]
    
    return cleaned_vocab

In [190]:
import json

def update_tokenizer_with_clean_vocab(new_tokens):
    old_tokenizer = deepcopy(labse.tokenizer)
    
    #conflicting_tokens = find_conflicting_tokens(new_tokens, old_tokenizer.vocab)
    #cleaned_vocab = remove_conflicting_tokens(conflicting_tokens, old_tokenizer.vocab)

    with open(f"{TKN_DIR_OLD}/tokenizer.json", "r", encoding='utf-8') as f:
        cfg = json.load(f)
    
    # Обновляем словарь в токенизаторе
    cfg['model']['vocab'] = cleaned_vocab

    with open(f"{TKN_DIR_NEW}/tokenizer.json", "w", encoding='utf-8') as f:
        json.dump(cfg, f, indent=False)

    with open(f"{TKN_DIR_NEW}/vocab.txt", "w", encoding='utf-8') as f:
        f.write('\n'.join(list(cfg['model']['vocab'].keys())))

    new_tokenizer = AutoTokenizer.from_pretrained(TKN_DIR_NEW)
    assert len(new_tokenizer.vocab) != 0
    # Добавляем новые токены
    new_tokens_to_add = list(set(new_tokens) - set(new_tokenizer.vocab.keys()))
    new_tokenizer.add_tokens(new_tokens_to_add)
    
    return new_tokenizer, new_tokens_to_add

In [327]:
def add_tokens_resave(vocab):
    new_tokenizer = deepcopy(labse.tokenizer)
    new_tokens = list(set(vocab) - set(new_tokenizer.vocab.keys()))
    num_tokens_added = new_tokenizer.add_tokens(new_tokens)

    with open(f"{TKN_DIR_OLD}/tokenizer.json", "r", encoding='utf-8') as f:
        cfg = json.load(f)
    
    cfg['model']['vocab'] = new_tokenizer.vocab
    
    with open(f"{TKN_DIR_NEW}/tokenizer.json", "w", encoding='utf-8') as f:
        json.dump(cfg, f, indent=False)

    with open(f"{TKN_DIR_NEW}/vocab.txt", "w", encoding='utf-8') as f:
        f.write('\n'.join(list(cfg['model']['vocab'].keys())))
    new_tokenizer = AutoTokenizer.from_pretrained(TKN_DIR_NEW, use_fast=False)
    print(f'Added {num_tokens_added} new tokens to tokenizer')
    return new_tokenizer, new_tokens

In [328]:
new_tokenizer, new_tokens = add_tokens_resave(list(new_wordpiece.get_vocab().keys())) #

Added 767 new tokens to tokenizer


In [187]:
def test_tokenizers(idx, old_tokenizer=old_tokenizer, new_tokenizer=new_tokenizer):
    source = data.source.loc[idx]
    target = data.target.loc[idx]
    print('Orig mansi:')
    print(target)
    #print()
    print('Old tokenizer on mansi:')
    l1 = old_tokenizer.convert_ids_to_tokens(old_tokenizer.encode(target))
    print(l1)
    #print()
    l11 = old_tokenizer.decode(old_tokenizer.encode(target)[1:-1])
    print('New tokenizer on mansi:')
    l2 = new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(target))
    print(l2)
    #print()
    l22 = new_tokenizer.decode(new_tokenizer.encode(target)[1:-1])
    print("Tokenized equally:", l1 == l2)
    is_equal_decode = l11 == l22 == target
    if not is_equal_decode:
        print('Decoded not equally')
        print('Decoded on old:', l11)
        print('Decoded on new:', l22)
    else:
        print('Decoded equally')
    print()
    print('Orig rus:')
    print(source)
    print('Old tokenizer on rus:')
    l1 = old_tokenizer.convert_ids_to_tokens(old_tokenizer.encode(source))
    print(l1)
    l11 = old_tokenizer.decode(old_tokenizer.encode(source)[1:-1])
    #print()
    print('New tokenizer on rus:')
    l2 = new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(source))
    print(l2)
    l22 = new_tokenizer.decode(new_tokenizer.encode(source)[1:-1])
    #print()
    print("Tokenized equally:", l1 == l2)
    is_equal_decode = l11 == l22 == source
    if not is_equal_decode:
        print('Decoded not equally')
        print('Decoded on old:', l11)
        print('Decoded on new:', l22)
    else:
        print('Decoded equally')

In [188]:
data[data.target.str.contains("ханищта")].index

Index([   61,    65,   135,   152,   178,   386,   393,   481,   491,  1136,
       ...
       75672, 75734, 76334, 76689, 76908, 76938, 77139, 77295, 77316, 77334],
      dtype='int64', length=2286)

In [298]:
print(tokenize(data.target.loc[152], old_tokenizer.vocab))

['Так', '##ви', 'па', '##лт', '##э', 'о', '##̄', '##нь', '##ща', '##ст', '##э', ',', 'ян', '##мал', '##тас', '##тэ', ',', '[UNK]', 'ос', 'та', '##̄', '##кса', '##рыг', '[UNK]', 'хан', '##ищ', '##тас', '##тэ', '.']


In [304]:
print(tokenize(data.target.loc[152], new_tokenizer.vocab))

['[CLS]', 'Так', '##ви', 'палт', '##э', 'о̄ньщ', '##аст', '##э', ',', 'янмал', '##тастэ', ',', 'рӯпит', '##а', '##ӈкв', 'ос', 'та̄', '##кса', '##рыг', 'о̄л', '##уӈкв', 'ханищ', '##тастэ', '.', '[SEP]']


In [307]:
new_tokenizer.decode(token2ids(tokenize(data.target.loc[152], new_tokenizer.vocab), new_tokenizer.vocab))

'[CLS] Такви палтэ о̄ньщастэ, янмалтастэ, рӯпитаӈкв ос та̄ксарыг о̄луӈкв ханищтастэ. [SEP]'

In [325]:
new_tokenizer2 = AutoTokenizer.from_pretrained(TKN_DIR_NEW, use_fast=False)

C:\Users\1645286\venvs\py310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [388]:
nas = []
for i in tqdm(range(2000)):
    if not new_tokenizer.encode(data.target.loc[i]) == token2ids(tokenize(data.target.loc[i], new_tokenizer.vocab), new_tokenizer.vocab):
        nas.append(i)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [389]:
nas[:5]

[1, 4, 6, 8, 13]

In [394]:
i = 4
data.target.loc[i]

'Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёрувлвылув'

In [401]:
'пӣлтал'[2:4]

'̄л'

In [408]:
'̄л' in new_tokenizer.vocab

False

In [395]:
print(tokenize(data.target.loc[i], new_tokenizer.vocab))

['[CLS]', 'Ман', 'ты', 'пи', '##̄л', '##тал', ',', 'весь', '##кат', 'хум', '##ию', '##в', 'н', '##эт', '##ху', '##нь', '##т', 'ат', 'ё', '##ру', '##в', '##лв', '##ылу', '##в', '[SEP]']


In [396]:
print(new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(data.target.loc[i])))

['[CLS]', 'Ман', 'ты', 'п', '##ӣ', '##лта', '##л', ',', 'весь', '##кат', 'хум', '##ию', '##в', 'н', '##эт', '##ху', '##нь', '##т', 'ат', 'ё', '##ру', '##в', '##лв', '##ылу', '##в', '[SEP]']


In [397]:
print(new_tokenizer.encode(data.target.loc[i]))
print(token2ids(tokenize(data.target.loc[i], new_tokenizer.vocab), new_tokenizer.vocab))

[101, 387098, 82546, 382840, 1007, 212162, 270774, 491731, 172342, 341765, 400044, 116046, 439312, 341134, 306762, 500300, 204659, 348463, 201819, 7925, 110251, 439312, 173324, 41862, 439312, 102]
[101, 387098, 82546, 132601, 409473, 376164, 491731, 172342, 341765, 400044, 116046, 439312, 341134, 306762, 500300, 204659, 348463, 201819, 7925, 110251, 439312, 173324, 41862, 439312, 102]


In [387]:
'##ӯр' in new_tokenizer.vocab

True

In [329]:
test_tokenizers(152, old_tokenizer, new_tokenizer)

Orig mansi:
Такви палтэ о̄ньщастэ, янмалтастэ, рӯпитаӈкв ос та̄ксарыг о̄луӈкв ханищтастэ.
Old tokenizer on mansi:
['[CLS]', 'Так', '##ви', 'па', '##лт', '##э', 'о', '##̄', '##нь', '##ща', '##ст', '##э', ',', 'ян', '##мал', '##тас', '##тэ', ',', '[UNK]', 'ос', 'та', '##̄', '##кса', '##рыг', '[UNK]', 'хан', '##ищ', '##тас', '##тэ', '.', '[SEP]']
New tokenizer on mansi:
['[CLS]', 'Так', '##ви', 'палт', '##э', 'о̄ньщ', '##аст', '##э', ',', 'янмал', '##тастэ', ',', 'рӯ', '##пита', '##ӈкв', 'ос', 'та̄', '##кса', '##рыг', 'о̄л', '##уӈкв', 'ханищ', '##тастэ', '.', '[SEP]']
Tokenized equally: False
Decoded not equally
Decoded on old: Такви палтэ о̄ньщастэ, янмалтастэ, [UNK] ос та̄ксарыг [UNK] ханищтастэ.
Decoded on new: Такви палтэ о̄ньщастэ, янмалтастэ, рӯпитаӈкв ос та̄ксарыг о̄луӈкв ханищтастэ.

Orig rus:
Он вырастил и воспитал мальчика, привив ему трудолюбие и твёрдый характер.
Old tokenizer on rus:
['[CLS]', 'Он', 'вы', '##расти', '##л', 'и', 'во', '##сп', '##ита', '##л', 'мальчика', ',', 

In [286]:
def encode_word(word, vocab):
    tokens = []
    c = 0
    while len(word) > 0:
        i = len(word)
        start_idx = 2 if word.startswith('##') else 0
        while i-start_idx > 0 and word[:i] not in vocab:
            #print(word[:i])
            i -= 1
        if i-start_idx == 0:
            return ['[UNK]']
        #print(i, word[:i])
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
        #print(word)
        c += 1
        if c > 10:
            break
    return tokens

In [302]:
def tokenize(texts, vocab):
    is_str = False
    if isinstance(texts, str):
        is_str = True
        texts = [texts]
    tokens = []
    for text in texts:
        pre_tokenize_result = old_tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
        pre_tokenized_text = [word for word, offset in pre_tokenize_result]
        tokens.append([encode_word(word, vocab) for word in pre_tokenized_text])
    tokens = [['[CLS]'] + sum(ts, []) + ['[SEP]'] for ts in tokens]
    return tokens if not is_str else tokens[0]

In [366]:
def token2ids(tokens, vocab):
    special = {
        '[PAD]': 0,
        '[UNK]': 100,
        "[CLS]": 101,
        "[SEP]": 102,
        "[MASK]": 103
    }
    return [special[token] if token in special else vocab[token] for token in tokens]

In [170]:
[new_wordpiece.id_to_token(idx) for idx in new_wordpiece.encode(data.target.loc[152]).ids]

['Та',
 '##кв',
 '##и',
 'палт',
 '##э',
 'о̄ньщ',
 '##ас',
 '##тэ',
 ',',
 'янмал',
 '##тастэ',
 ',',
 'рӯпит',
 '##а',
 '##ӈкв',
 'ос',
 'та̄',
 '##кс',
 '##ар',
 '##ыг',
 'о̄л',
 '##уӈкв',
 'ханищ',
 '##тастэ',
 '.']

# train new tokenizer

In [12]:
from tokenizers.trainers import WordPieceTrainer
from tokenizers import models, pre_tokenizers, decoders, Tokenizer
import json
import os

In [8]:
new_wordpiece = Tokenizer(models.WordPiece())
new_wordpiece.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
trainer = WordPieceTrainer(
    vocab_size=1200,
    min_frequency=30,
    show_progress=True,
    continuing_subword_prefix='##'
)

In [9]:
new_wordpiece.train_from_iterator(all_corpus, trainer=trainer, length=len(all_corpus))

In [10]:
[el for el in sorted(new_wordpiece.get_vocab(), key=lambda x: len(x), reverse=True) if 'ханищта' in el]

['ханищтах', 'ханищтан']

In [11]:
sorted(new_wordpiece.get_vocab(), key=lambda x: len(x), reverse=True)

['э̄лумхо̄лас',
 'рӯпитаӈкве',
 'Саранпа̄выл',
 'рӯпитэ̄гыт',
 '##лумхо̄лас',
 'потыртасыт',
 '##таве̄сыт',
 '##щхӣпыӈ',
 'хо̄талэ̄т',
 'ва̄руӈкве',
 '##има̄гыс',
 'на̄врамыт',
 'ёхталасыт',
 'Александр',
 'ва̄рмалит',
 'ня̄врамыт',
 'ма̄хманув',
 '##е̄ккар',
 '##а̄врам',
 '##лэ̄гыт',
 '##луӈкве',
 'ва̄рмаль',
 'потыртас',
 '##хо̄лас',
 '##па̄выл',
 'рӯпитан',
 'о̄луӈкве',
 'нэ̄пакыт',
 'ма̄ньлат',
 'пӯльниц',
 '##туӈкве',
 'Мансийск',
 'ханищтах',
 '##тыяныл',
 '##тэ̄гыт',
 'о̄вылтах',
 'ищхӣпыӈ',
 'та̄наныл',
 '##таӈкве',
 'Ха̄льӯс',
 'рӯпитас',
 '##ве̄сыт',
 'о̄лэ̄гыт',
 'ханищтан',
 'халанылт',
 'округувт',
 'пормасыт',
 '##лаӈкве',
 '##тыглас',
 '##рищит',
 '##э̄гыт',
 '##анылт',
 '##лавес',
 'ва̄рмал',
 '##о̄рум',
 '##ияныл',
 '##тасум',
 'ма̄ньщи',
 'о̄влэ̄т',
 'ле̄ккар',
 'на̄врам',
 '##уӈкве',
 'ня̄врам',
 '##а̄гыл',
 '##е̄гыт',
 'Алексан',
 'са̄всыр',
 'рӯпата',
 '##тавес',
 'па̄вылт',
 '##̄врам',
 'мӯйлуп',
 '##таӈкв',
 'рӯпиты',
 '##ласыт',
 '##тавет',
 'ёмащакв'

In [35]:
TKN_DIR_OLD = '../data/tokenizers/labse_tokenizer'
TKN_DIR_NEW = '../data/tokenizers/tokenizer_v1'

In [36]:
def add_tokens_resave(vocab):
    new_tokenizer = deepcopy(labse.tokenizer)
    new_tokens = list(set(vocab) - set(new_tokenizer.vocab.keys()))
    num_tokens_added = new_tokenizer.add_tokens(new_tokens)

    with open(f"{TKN_DIR_OLD}/tokenizer.json", "r", encoding='utf-8') as f:
        cfg = json.load(f)
    
    cfg['model']['vocab'] = new_tokenizer.vocab
    
    with open(f"{TKN_DIR_NEW}/tokenizer.json", "w", encoding='utf-8') as f:
        json.dump(cfg, f, indent=False)

    with open(f"{TKN_DIR_NEW}/vocab.txt", "w", encoding='utf-8') as f:
        f.write('\n'.join(list(cfg['model']['vocab'].keys())))
    new_tokenizer = AutoTokenizer.from_pretrained(TKN_DIR_NEW, use_fast=False)
    print(f'Added {num_tokens_added} new tokens to tokenizer')
    #os.remove(TKN_DIR_NEW)
    return new_tokenizer, new_tokens

In [37]:
new_tokenizer, new_tokens = add_tokens_resave(list(new_wordpiece.get_vocab().keys())) #

Added 557 new tokens to tokenizer


In [39]:
new_tokenizer = AutoTokenizer.from_pretrained('../data/tokenizers/tokenizer_v1', use_fast=False)

In [40]:
def test_tokenizers(idx, old_tokenizer=labse.tokenizer, new_tokenizer=new_tokenizer):
    source = data.source.loc[idx]
    target = data.target.loc[idx]
    print('Orig mansi:')
    print(target)
    #print()
    print('Old tokenizer on mansi:')
    l1 = old_tokenizer.convert_ids_to_tokens(old_tokenizer.encode(target))
    print(l1)
    #print()
    l11 = old_tokenizer.decode(old_tokenizer.encode(target)[1:-1])
    print('New tokenizer on mansi:')
    l2 = new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(target))
    print(l2)
    #print()
    l22 = new_tokenizer.decode(new_tokenizer.encode(target)[1:-1])
    print("Tokenized equally:", l1 == l2)
    is_equal_decode = l11 == l22 == target
    if not is_equal_decode:
        print('Decoded not equally')
        print('Decoded on old:', l11)
        print('Decoded on new:', l22)
    else:
        print('Decoded equally')
    print()
    print('Orig rus:')
    print(source)
    print('Old tokenizer on rus:')
    l1 = old_tokenizer.convert_ids_to_tokens(old_tokenizer.encode(source))
    print(l1)
    l11 = old_tokenizer.decode(old_tokenizer.encode(source)[1:-1])
    #print()
    print('New tokenizer on rus:')
    l2 = new_tokenizer.convert_ids_to_tokens(new_tokenizer.encode(source))
    print(l2)
    l22 = new_tokenizer.decode(new_tokenizer.encode(source)[1:-1])
    #print()
    print("Tokenized equally:", l1 == l2)
    is_equal_decode = l11 == l22 == source
    if not is_equal_decode:
        print('Decoded not equally')
        print('Decoded on old:', l11)
        print('Decoded on new:', l22)
    else:
        print('Decoded equally')

In [41]:
test_tokenizers(6666)

Orig mansi:
О̄йка сӯнсы воссыг нэ̄матыр ат хультыс, Тыстылэ нупыл ла̄ви: «Ха-та, Тыстыл, ёмщакв уральта̄хтэн, э̄рыӈ тот иӈ олн хультыс?
Old tokenizer on mansi:
['[CLS]', 'О', '##̄', '##йка', 'су', '##̄', '##нсы', 'во', '##сс', '##ыг', 'н', '##э', '##̄', '##мат', '##ыр', 'ат', 'ху', '##льт', '##ыс', ',', 'Ты', '##сты', '##лэ', 'ну', '##пы', '##л', 'ла', '##̄', '##ви', ':', '«', 'Ха', '-', 'та', ',', 'Ты', '##сты', '##л', ',', 'ём', '##ща', '##к', '##в', 'ура', '##льт', '##а', '##̄', '##хт', '##эн', ',', '[UNK]', 'тот', '[UNK]', 'ол', '##н', 'ху', '##льт', '##ыс', '?', '[SEP]']
New tokenizer on mansi:
['[CLS]', 'О', '##̄й', '##ка', 'с', '##ӯ', '##нсы', 'вос', '##сыг', 'нэ̄м', '##аты', '##р', 'ат', 'хульт', '##ыс', ',', 'Ты', '##сты', '##лэ', 'нупыл', 'ла̄ви', ':', '«', 'Ха', '-', 'та', ',', 'Ты', '##сты', '##л', ',', 'ёмщакв', 'ураль', '##та', '##̄', '##хт', '##эн', ',', 'э̄р', '##ыӈ', 'тот', 'иӈ', 'олн', 'хульт', '##ыс', '?', '[SEP]']
Tokenized equally: False
Decoded not equally
Decode

# negatives

In [35]:
negative_rus = []
negative_mans = []


for dict_scores in tqdm(data['similarities']):
    
    filtered_items = {k: v for k, v in dict_scores.items() if 0.89 <= v < 0.99}
    
    if len(filtered_items) > 0:
        max_id = max(filtered_items, key=filtered_items.get)
        negative_example_rus = data['source'].iloc[max_id]
        negative_example_mans = data['target'].iloc[max_id]
    else:
        negative_example_rus = None
        negative_example_mans = None
    
    negative_rus.append(negative_example_rus)
    negative_mans.append(negative_example_mans)
    
data['negative_examples_mans'] = negative_mans
data['negative_examples_rus'] = negative_rus

  0%|          | 0/77374 [00:00<?, ?it/s]

In [36]:
data.negative_examples_mans.value_counts(dropna=False)

negative_examples_mans
None                                                                                                            6951
Та̄наныл янытлаӈкв округ кӯщай нэ̄ Наталья Комарова тув ёхталас.                                                 14
Тав колхозт, тувыл совхозт рӯпитас.                                                                              13
Хоса минас, ва̄ти минас, колэ̄н ёхтыс.                                                                            12
Хӯнь во̄ртолнут ӯй хӯл ка̄т ла̄ква - во̄ськасам та̄гмахтум.                                                    12
                                                                                                                ... 
Тав тамле потыртахтын ва̄рмаль мир ёт ос ӯйхулт ёт ва̄рыс.                                                        1
Ной пы̄ганэ са̄в ня̄врам о̄сьсыт.                                                                                  1
Таима̄гыс То̄рум А̄син та̄н са̄литаве̄сыт